# Torch Segmentation Model with Lab Dataset

Based off of:

https://towardsdatascience.com/train-neural-net-for-semantic-segmentation-with-pytorch-in-50-lines-of-code-830c71a6544f
https://github.com/sagieppel/Train-Semantic-Segmentation-Net-with-Pytorch-In-50-Lines-Of-Code 

MIT License - use available for commercial use

In [1]:
import os
import numpy as np
import cv2
import torchvision.models.segmentation
import torch
import torchvision.transforms as tf
import random

Learning_Rate=1e-5
width=height=800 # image width and height
batchSize=3

In [2]:
TrainFolder="Data/LabPicsV1/Simple/Train/"
ListImages=os.listdir(os.path.join(TrainFolder, "Image"))

In [3]:
transformImg=tf.Compose([tf.ToPILImage(),tf.Resize((height,width)), tf.ToTensor(),tf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

transformAnn=tf.Compose([tf.ToPILImage(),tf.Resize((height,width)), tf.ToTensor()])

In [7]:
# Read custom image set into cache so that we don't have to load on every single run
custom_image_loaded = []
custom_masks_loaded = []

def loadImages(img_arr, mask_arr, full_path_images, full_path_masks): # full path is the parent folder of images
    data = os.listdir(full_path_images)
    for file in data:
        img_single_path = os.path.join(full_path_images, file)
        mask_single_path = os.path.join(full_path_masks, file)

        img_arr.append(cv2.imread(img_single_path))
        temp_mask = cv2.imread(os.path.join(mask_single_path.replace("jpeg","png").replace("jpg","png"))).max(axis=2) # perhaps switch custom_masks[idx] with custom_imgs[idx] to match image
        temp_mask[temp_mask > 0] = 1
        mask_arr.append(temp_mask)

second_train_folder = "../../Skin_Anatomical_Image_Dataset/simple_image_config2"
full_path_images = second_train_folder + "/images"
full_path_masks = second_train_folder + "/masks"
loadImages(custom_image_loaded, custom_masks_loaded, full_path_images, full_path_masks)

print("Length of images and masked images arrays: ", len(custom_image_loaded), ", ", len(custom_masks_loaded))

Length of images and masked images arrays:  77 ,  77


In [8]:
probability = 0.25

cust_img_path = os.path.join(second_train_folder, "images")
cust_mask_path = os.path.join(second_train_folder, "masks")

custom_imgs = os.listdir(cust_img_path)
custom_masks = os.listdir(cust_mask_path)

# Get length of directory
len_custom_masks = len(custom_masks)
print("Number of files in custom directory: ", len_custom_masks)

def ReadRandomImage(show=False):

    # Probabilistically read in an image that is from our custom dataset
    if random.random() < probability:
        idx=np.random.randint(0,len_custom_masks)
        Filled = None

        # Replace loading by our cache
        # Img=cv2.imread(os.path.join(cust_img_path,custom_imgs[idx]))
        # Vessel =  cv2.imread(os.path.join(cust_mask_path, custom_imgs[idx].replace("jpeg","png").replace("jpg","png"))).max(axis=2) # perhaps switch custom_masks[idx] with custom_imgs[idx] to match image
        # Vessel[Vessel > 0] = 1

        Img = custom_image_loaded[idx]
        Vessel = custom_masks_loaded[idx]        
    else:
        idx=np.random.randint(0,len(ListImages)) # Pick random image   
        Img=cv2.imread(os.path.join(TrainFolder, "Image",ListImages[idx]))  
        Filled =  cv2.imread(os.path.join(TrainFolder,   "Semantic/16_Filled", ListImages[idx].replace("jpg","png")),0)    
        Vessel =  cv2.imread(os.path.join(TrainFolder, "Semantic/1_Vessel", ListImages[idx].replace("jpg","png")),0)

    if show:
        print("Image path: ", str(os.path.join(TrainFolder, "Image",ListImages[idx])))
        # show the image, provide window name first
        cv2.imshow('Img', Img)
        cv2.waitKey(0)
        # cv2.imshow('Filled', Filled)
        # cv2.waitKey(0)
        cv2.imshow('Vessel', Vessel)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        # print("Filled: ", Filled)
        print("Vessel num: ", np.count_nonzero(Vessel), "/", Vessel.shape, ", range of values: [", np.min(Vessel), ", ", np.max(Filled), "]")
        print("numNan: ", sum(~np.isnan(Vessel)))
        print("Vessel: ", Vessel)

    AnnMap = np.zeros(Img.shape[0:2],np.float32)
    
    if Vessel is not None:  
        AnnMap[ Vessel == 1 ] = 1    
    if Filled is not None:  
        AnnMap[ Filled  == 1 ] = 2

    Img=transformImg(Img)
    AnnMap=transformAnn(AnnMap)

    return Img,AnnMap, Vessel

Number of files in custom directory:  77


In [9]:
# Print test images form dataset
Img,AnnMap, Vessel = ReadRandomImage(show=True)

Image path:  Data/LabPicsV1/Simple/Train/Image/IMG_20190124_014107.jpg


error: OpenCV(4.3.0) /tmp/pip-req-build-fcxkfqxp/opencv/modules/highgui/src/window.cpp:376: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


In [10]:
#--------------Load batch of images-----------------------------------------------------
def LoadBatch(): # Load batch of images
    images = torch.zeros([batchSize,3,height,width])
    ann = torch.zeros([batchSize, height, width])

    for i in range(batchSize):
        images[i],ann[i],_=ReadRandomImage()
    
    return images, ann

In [11]:
#--------------Load and set net and optimizer-------------------------------------
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
Net = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=True) # Load net
Net.classifier[4] = torch.nn.Conv2d(256, 3, kernel_size=(1, 1), stride=(1, 1)) # Change final layer to 3 classes
Net=Net.to(device)
optimizer=torch.optim.Adam(params=Net.parameters(),lr=Learning_Rate) # Create adam optimizer

/home/carwyn/anaconda3/envs/hypertrophy/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/home/carwyn/anaconda3/envs/hypertrophy/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
#----------------Train--------------------------------------------------------------------------
for itr in range(25001): # Training loop
   images,ann=LoadBatch() # Load taining batch
   images=torch.autograd.Variable(images,requires_grad=False).to(device) # Load image
   ann = torch.autograd.Variable(ann, requires_grad=False).to(device) # Load annotation
   Pred=Net(images)['out'] # make prediction
   Net.zero_grad()
   criterion = torch.nn.CrossEntropyLoss() # Set loss function
   Loss=criterion(Pred,ann.long()) # Calculate cross entropy loss
   Loss.backward() # Backpropogate loss
   optimizer.step() # Apply gradient descent change to weight
   seg = torch.argmax(Pred[0], 0).cpu().detach().numpy()  # Get  prediction classes
   print(itr,") Loss=",Loss.data.cpu().numpy())
   
   if itr % 1000 == 0: #Save model weight once every 60k steps permenant file
        print("Saving Model" +str(itr) + ".torch")
        torch.save(Net.state_dict(),  "models/full_custom_dataset_p_0.25_" + str(itr) + ".torch")

0 ) Loss= 1.0242001
Saving Model0.torch
1 ) Loss= 0.9636108
2 ) Loss= 1.0618408
3 ) Loss= 0.92679566
4 ) Loss= 0.8699037
5 ) Loss= 0.9843594
6 ) Loss= 0.93880635
7 ) Loss= 0.8854367
8 ) Loss= 0.87661207
9 ) Loss= 0.9544825
10 ) Loss= 0.8528353
11 ) Loss= 0.9075309
12 ) Loss= 0.88045806
13 ) Loss= 0.89023393
14 ) Loss= 0.87255377
15 ) Loss= 0.91094375
16 ) Loss= 0.8651805
17 ) Loss= 0.8479434
18 ) Loss= 0.82564455
19 ) Loss= 0.8595954
20 ) Loss= 0.79570013
21 ) Loss= 0.8869397
22 ) Loss= 0.8405006
23 ) Loss= 0.834586
24 ) Loss= 0.8812895
25 ) Loss= 0.89322525
26 ) Loss= 0.7959807
27 ) Loss= 0.85650724
28 ) Loss= 0.8136668
29 ) Loss= 0.880726
30 ) Loss= 0.81182736
31 ) Loss= 0.8391388
32 ) Loss= 0.84078
33 ) Loss= 0.7962342
34 ) Loss= 0.7919302
35 ) Loss= 0.81724817
36 ) Loss= 0.82004833
37 ) Loss= 0.7866733
38 ) Loss= 0.77584404
39 ) Loss= 0.7740277
40 ) Loss= 0.80170083
41 ) Loss= 0.8262077
42 ) Loss= 0.76572
43 ) Loss= 0.7710629
44 ) Loss= 0.8422011
45 ) Loss= 0.73736954
46 ) Loss= 0.